# Total Factor Productivity

This notebook uses historical data on Total Factor Productivity compiled by John Fernald and downloaded from the San Francisco Fed's website at the following link: http://www.frbsf.org/economic-research/indicators-data/total-factor-productivity-tfp/

More documentation on the methodology can be found on the website as well as the accompanying Excel spreadsheet. The data were downloaded on September 28, 2020, and run from 1947:Q2 to 2020:Q2. We only read data through 2019:Q4 to avoid the pandemic period.

The program extends the Fernald series forward in time using projections made by Robert J. Gordon in his paper "Secular Stagnation on the Supply Side: U.S. Productivity Growth in the Long Run" (link: http://economics.weinberg.northwestern.edu/robert-gordon/files/RescPapers/SecularStagnation.pdf). Gordon estimates that total factor productivity will grow at a annual rate of 1.1 percent per year through 2032, a rate that we extend beyond that date.

The program also extends the Fernald series back in time using an informed guess of average technology growth in earlier decades. The rate chosen---2 percent---is in the middle of the range of estimates surveyed by Shackleton (2013).

Of note, the TFP growth measure constructed by Fernald (2014) nets out the contribution of labor quality growth, which averages about 0.25 percentage point on an annualized basis. By contrast, the estimates of Shackleton (2013) and Gordon (2015) conflate labor quality growth with TFP growth. To ensure consistency across data sources and with our model, which does not feature labor quality growth, we add back the contribution of labor quality growth to Fernald's estimates. 

The program also exports Fernald's estimate of the log change in labor quality, $\Delta log(HQ_{t})$. We extend the series backward and forward in time using the average growth for the Fernald (2014) series. This series-wide average is close to that observed in the first and last 10 years, though wider swings can be observed in the decades in between. We used the resulting series in an unpublished exploration of the effect of including growth in human capital in the model.

Finally, our model does not seek to explain high-frequency movements in the equilibrium real rate, GDP, and other aggregate variables. For this reason, we feed our model with a smoothed version of the quarterly (log) TFP growth series. To create this smoothed series, we first convert our extended Fernald TFP growth series to levels and then HP filter the log of the series to extract a smooth trend. We finally convert the smoothed level series back to growth rates (measured in log changes).

__References__ 

John G. Fernald, "A Quarterly, Utilization-Adjusted Series on Total Factor Productivity." FRBSF Working Paper 2012--19 (updated March 2014).

Robert J. Gordon, "Secular Stagnation on the Supply Side: U.S. Productivity Growth in the Long Run." *Digiworld Economic Journal*, no. 100 (4-th quarter 2015).

Robert Shackleton, "Total Factor Productivity Growth in Historical Perspective." Working Paper 2013--01, Congressional Budget Office, Washington D.C. (March 2013).

In [ ]:
# Read in the data
using CSV, DataFrames;
tfp_q = CSV.read("RawData/quarterly_tfp.csv", DataFrame);

imax = 292;   # Last period read in spreadsheet (imax = 292 -> 2019:Q4)
jmin = 190;   # Sample period corresponding to first period in spreadsheet (190 = 1947:Q2)
jmax = 480;   # Sample period corresponding to last period in spreadsheet (480 = 2019:Q4)


# Allocating historical data to array
periods_q = 1900.125:0.25:2400       # Period in model simulation
dTFP = NaN*ones(length(periods_q))

# Adding back labor quality that Fernald had removed
dTFP[jmin:jmax] = tfp_q.dtfp_util[2:imax]+(1.0.-tfp_q.alpha[2:imax]).*tfp_q.dLQ[2:imax];

# Calculating mean contribution of labor quality to TFP
mean_dLQ=sum((1.0.-tfp_q.alpha[2:imax]).*tfp_q.dLQ[2:imax])/length(dTFP[jmin:jmax])
println("Mean contribution of labor quality growth to annualized quarterly technology growth: " * string(mean_dLQ) * " p.p.")

# TFP growth assumptions before and after historical sample
# Contrary to Fernald's growth statistics, which are in log changes multiplied by 400,
# the annual growth rates of Shackelton (2013) and Gordon (2015) are in standard annualized percent.
# Accordingly, they need to be converted to match the units in Fernald (2014).
dTFP[1:(jmin-1)] .= 100*log(1+2.0/100)       # 1900:Q1 through 1947:Q1
dTFP[(jmax+1):end] .= 100*log(1+1.1/100)     # 2016:Q2 onward

# Constructing a TFP series in levels
TFP = ones(length(periods_q))
TFP[1] = 1.0
for i=2:length(dTFP)
    TFP[i] = TFP[i-1]*exp(dTFP[i]/400);
end

# Writing the data for plotting 
using DelimitedFiles
writedlm("CleanData/TFP_Q.csv", TFP, ',');

# HP-filter the (log of) TFP series
include("hpfilter.jl")
logTFP_hp = hpfilter(log.(TFP),25000);

# Exporting quarterly series of log-change in TFP to CSV file
dlogTFP_q = Array{Float64}(undef, length(dTFP))
dlogTFP_q[2:end]=logTFP_hp[2:end]-logTFP_hp[1:(end-1)]
dlogTFP_q[1] = dlogTFP_q[2]
writedlm("CleanData/dlogtfp_Q.csv", dlogTFP_q, ',')

# Exporting yearly series of log-change in TFP to CSV file
dlogTFP_y = Array{Float64}(undef, length(dTFP))
dlogTFP_y[5:end] = logTFP_hp[5:end]-logTFP_hp[1:end-4]
dlogTFP_y = dlogTFP_y[4:4:end]
dlogTFP_y[1] = dlogTFP_y[2]
writedlm("CleanData/dlogtfp_Y.csv", dlogTFP_y, ',');

In [ ]:
# Extracting an (annualized) labor quality (LQ) growth series
dLQ = NaN*ones(length(periods_q))
dLQ[jmin:jmax] = tfp_q.dLQ[2:imax]
dLQ[1:(jmin-1)] .= sum(dLQ[jmin:jmax])/length(dLQ[jmin:jmax])    
dLQ[(jmax+1):end] .= sum(dLQ[jmin:jmax])/length(dLQ[jmin:jmax])

# Building level
LQ = ones(length(dLQ))
for i=2:length(dLQ)
    LQ[i] = LQ[i-1]*exp(dLQ[i]/400);
end

# Writing the level data for plotting 
writedlm("CleanData/LQ_Q.csv", LQ, ',');

# HP-filter the (log of) LQ series
logLQ_hp = hpfilter(log.(LQ),25000);

# Exporting quarterly series of log-change in LQ to CSV file
dlogLQ_q = Array{Float64}(undef, length(dLQ))
dlogLQ_q[2:end]=logLQ_hp[2:end]-logLQ_hp[1:(end-1)]
dlogLQ_q[1] = dlogLQ_q[2]
writedlm("CleanData/dlogLQ_Q.csv", dlogLQ_q, ',')

# Exporting yearly series of log-change in LQ to CSV file
dlogLQ_y = Array{Float64}(undef, length(dLQ))
dlogLQ_y[5:end] = logLQ_hp[5:end]-logLQ_hp[1:end-4]
dlogLQ_y = dlogLQ_y[4:4:end]
dlogLQ_y[1] = dlogLQ_y[2]
writedlm("CleanData/dlogLQ_Y.csv", dlogLQ_y, ',');

In [ ]:
# Inspecting the LQ growth series
using Plots
plot(periods_q[190:465],dLQ[190:465],title = "Human quality (original)",label="",ylabel="annualized percent")
myper=120:500
plot(periods_q[myper],400*dlogLQ_q[myper],title = "Human quality (smoothed)",label="",ylabel="annualized percent")

In [ ]:
# Changing permissions
run(`chmod 664 CleanData/TFP_Q.csv`);
run(`chmod 664 CleanData/dlogtfp_Q.csv`);
run(`chmod 664 CleanData/dlogtfp_Y.csv`);
run(`chmod 664 CleanData/LQ_Q.csv`);
run(`chmod 664 CleanData/dlogLQ_Q.csv`);
run(`chmod 664 CleanData/dlogLQ_Y.csv`);

The next set of lines were written in response to a referee who was interested in the difference between Fernald's human capital index series and the one reported in the Penn World Table. The output is not used in the main modeling files. The sample period corresponds to an earlier sample period for the Fernald data pull.

In [ ]:
# Reading the index of human capital per person for the United States
# https://fred.stlouisfed.org/series/HCIYISUSA066NRUG#0
IHCUSA_data = readdlm("RawData/FRED_HCIYISUSA066NRUG.csv", ',', header=true)[1][:,2];
IHCUSA_year = 1950.0:2017 .+ 0.5;
mydata = [LQ[201:4:464]./LQ[201] IHCUSA_data[1:end-2]./IHCUSA_data[1]] 

# Plotting and inspecting
using Plots
p50=plot(IHCUSA_year[1:end-2],mydata, 
    title="Index of labor quality",
    label=["Fernald (2014)" "Penn World Tables"],
    legend =:topleft,
    color=[:blue :black],   
    linestyle = [:solid :dash],
    linewidth = [4 4]
)
display(p50)

# Interpolating data to quarterly frequency
include("ordernorep.jl")
include("spline_cubic.jl")
LQPWT = NaN*ones(length(periods_q))
LQPWT[202:462] = exp.(spline_cubic(IHCUSA_year,log.(IHCUSA_data),periods_q[202:462],0,0))
tmp=(log(IHCUSA_data[end]/IHCUSA_data[1]))/(length(IHCUSA_data)-1)
for i=201:-1:1
    LQPWT[i] = LQPWT[i+1]*exp(-tmp/4);
end
for i=463:length(LQPWT)
    LQPWT[i] = LQPWT[i-1]*exp(tmp/4);
end

# HP-filter the (log of) LQ series
logLQPWT_hp = hpfilter(log.(LQPWT),25000);

# Exporting quarterly series of log-change in LQ to CSV file
dlogLQPWT_q = Array{Float64}(undef, length(dLQ))
dlogLQPWT_q[2:end]=logLQPWT_hp[2:end]-logLQPWT_hp[1:(end-1)]
dlogLQPWT_q[1] = dlogLQPWT_q[2]
writedlm("CleanData/dlogLQPWT_Q.csv", dlogLQPWT_q, ',')
run(`chmod 664 CleanData/dlogLQPWT_Q.csv`);